# **Setores Industriais Canadá**

## *- Importando Bibliotecas*

In [1]:
import re
import pandas as pd
from io import StringIO

## *- Extraindo Dados*

In [2]:
with open("Dados/Landing/1410035501-noSymbol.csv", "r") as file:
    conteudo = file.read()

    texto_csv = ""

    conteudo_linhas = conteudo.split("\n")
    conteudo_linhas = conteudo_linhas[11:32]
    del conteudo_linhas[1]

    colunas = conteudo_linhas[0]

    lista_meses = colunas.split(",")
    meses = len(lista_meses[1:])

    for i, linha in enumerate(conteudo_linhas):

        valores = linha.split('"')

        valores = valores[1::2]

        valor_final = ";".join(valores)

        texto_csv += f"{valor_final}\n"


# *- Transformando Dados*

In [ ]:
arquivo_csv = StringIO(texto_csv)

df = pd.read_csv(arquivo_csv, sep=";")

display(df)

In [ ]:
print(df.columns)

In [5]:
#Renomeando a primeira coluna
df = df.rename(columns={"North American Industry Classification System (NAICS) 6": "North American Industry Classification System (NAICS)"})

In [ ]:
first = df.columns[0]

df[first] = df[first].apply(lambda x: re.sub(r"\d", "", x))

display(df[first])

In [ ]:
display(df.head())

In [ ]:


df.iloc[:,1:] = df.iloc[:,1:].map(lambda cel: re.sub(",","", cel))
df.iloc[:,1:] = df.iloc[:,1:].astype(float)



display(df)

In [ ]:
# Traduzindo as colunas
column_translation_dict = {
    'North American Industry Classification System (NAICS)': 'Sistema de Classificação de Indústrias da América do Norte (SCIAN)',
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}


def traduzir_coluna(column):
    for eng, ptbr in column_translation_dict.items():
        if eng in column:
            column = column.replace(eng, ptbr)
    return column


df.columns = df.columns.map(lambda column: column.strip())
df.columns = df.columns.map(traduzir_coluna)

display(df)

In [ ]:
# Traduzindo a primeira coluna
first = df.columns[0]

cell_translation_dict = {
    "Total employed, all industries": "Total empregado, todas as indústrias",
    "Goods-producing sector": "Setor de produção de bens",
    "Agriculture": "Agricultura",
    "Forestry, fishing, mining, quarrying, oil and gas": "Silvicultura, pesca, mineração, extração de pedreiras, petróleo e gás",
    "Utilities": "Serviços públicos",
    "Construction": "Construção",
    "Manufacturing": "Manufatura",
    "Services-producing sector": "Setor de produção de serviços",
    "Wholesale and retail trade": "Comércio atacadista e varejista",
    "Transportation and warehousing": "Transporte e armazenamento",
    "Finance, insurance, real estate, rental and leasing": "Finanças, seguros, imóveis, aluguel e leasing",
    "Professional, scientific and technical services": "Serviços profissionais, científicos e técnicos",
    "Business, building and other support services": "Serviços de negócios, construção e outros serviços de apoio",
    "Educational services": "Serviços educacionais",
    "Health care and social assistance": "Cuidados com a saúde e assistência social",
    "Information, culture and recreation": "Informação, cultura e recreação",
    "Accommodation and food services": "Acomodações e serviços de alimentação",
    "Other services (except public administration)": "Outros serviços (exceto administração pública)",
    "Public administration": "Administração pública"
}


def traduzir_cell(cell):
    for eng, ptbr in cell_translation_dict.items():
        if eng in cell:
            cell = cell.replace(eng,ptbr)
    return cell


df[first] = df[first].map(lambda cell: cell.strip())
df[first] = df[first].map(traduzir_cell)

display(df)

In [11]:
df.to_csv("Dados/Tratados/DadosSetoresCanada.csv", sep=";", index=False)

# *- Loading na S3*

In [12]:
import os
import boto3
from dotenv import load_dotenv


# Carregar variáveis
load_dotenv(r"credentials/.env")

# Configure as cresdencias da AWS
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Crie um client S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Carregue o arquivo para o S3
s3.upload_file(
    Filename='Dados/Tratados/DadosSetoresCanada.csv',
    Bucket='projetofinal-consuelo',
    Key='DadosSetoresCanada.csv'
)

# **Setores e Subsetores NAICS (2020-2023)**

## *- Importando Bibliotecas*

In [13]:
from dotenv import load_dotenv
import pandas as pd
import boto3
import wget
import ssl
import os

## *- Extração*

In [14]:
# Criando diretório (caso não exista)

try:
    os.mkdir("Dados")
    os.mkdir("Dados/Landing")
except:
    pass

In [ ]:
ssl._create_default_https_context = ssl._create_stdlib_context

url = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/ca165fad-4f16-48c7-808e-8b5222bc4182/download/qcew-2020-2022.csv'
wget.download(url, 'Dados/Landing/qcew2020-2022.csv')

url = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/119eef38-3b59-499f-8f7c-9bea4768469d/download/qcew-2023.csv'
wget.download(url, "Dados/Landing/qcew2023.csv")

## *- Transformação*

### -- Carregamento dos Dados

In [16]:
df2022 = pd.read_csv("Dados/Landing/qcew2020-2022.csv")
df2023 = pd.read_csv("Dados/Landing/qcew2023.csv")

### *-- Traduzindo Colunas*

In [17]:
colunas_traduzidas = {
    'Area Type': 'Tipo de Área',
    'Area Name': 'Nome da Área',
    'Year': 'Ano',
    'Time Period': 'Período de Tempo',
    'Ownership': 'Propriedade',
    'NAICS Level': 'Nível NAICS',
    'NAICS Code': 'Código NAICS',
    'Industry Name': 'Nome da Indústria',
    'Establishments': 'Estabelecimentos',
    'Average Monthly Employment': 'Emprego Médio Mensal',
    '1st Month Emp': 'Emprego no 1º Mês',
    '2nd Month Emp': 'Emprego no 2º Mês',
    '3rd Month Emp': 'Emprego no 3º Mês',
    'Total Wages (All Workers)': 'Salários Totais (Todos os Trabalhadores)',
    'Average Weekly Wages': 'Salário Médio Semanal'
}


df2022.rename(columns=colunas_traduzidas, inplace = True)
df2023.rename(columns=colunas_traduzidas, inplace = True)

In [ ]:
display(df2022.head())
print("\n-------------------\n")
display(df2023.head())

### *-- Padronizando Tipos*

In [19]:
#Primeiro transformando para inteiro para tirar o ponto flutuante
df2023['Nível NAICS'] = df2023['Nível NAICS'].astype(pd.Int64Dtype())
df2022['Nível NAICS'] = df2022['Nível NAICS'].astype(pd.Int64Dtype())

#Transformando para string
df2023['Nível NAICS'] = df2023['Nível NAICS'].astype(pd.StringDtype())
df2022['Código NAICS'] = df2022['Código NAICS'].astype(pd.StringDtype())


#Transformando Salário Médio em float
df2023['Salário Médio Semanal'] = df2023['Salário Médio Semanal'].astype(float)
df2022['Salário Médio Semanal'] = df2022['Salário Médio Semanal'].astype(float)

### *-- Juntando Tabelas*

In [20]:
df = pd.concat([df2022, df2023], ignore_index=True)

In [ ]:
display(df)

### *-- Separando Períodos Anuais dos Trimestrais*

In [22]:
df_anual = df[df['Período de Tempo'] == 'Annual']
df_trimestral = df[df['Período de Tempo'] != 'Annual']

df = df_trimestral # Backup código

In [ ]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

### *-- Traduzindo Valores*

In [24]:
# print(set(df["Período de Tempo"]))
# print(set(df["Tipo de Área"]))
# print(set(df["Nome da Área"]))
# print(set(df["Propriedade"]))


# Pedindo tradução ao GPT

temp_dict = {
    '1st Qtr': 'Trimestre 1',
    '2nd Qtr': 'Trimestre 2',
    '3rd Qtr': 'Trimestre 3',
    '4th Qtr': 'Trimestre 4',
    'Annual': 'Anual'
}


tipo_dict = {
    'United States': 'Estados Unidos',
    'California - Statewide': 'Califórnia - Estado',
    'County': 'Condado'
}

nome_dict = {
    'Marin County': 'Condado de Marin',
    'California': 'Califórnia',
    'Nevada County': 'Condado de Nevada',
    'Siskiyou County': 'Condado de Siskiyou',
    'Humboldt County': 'Condado de Humboldt',
    'Santa Barbara County': 'Condado de Santa Bárbara',
    'Fresno County': 'Condado de Fresno',
    'Kern County': 'Condado de Kern',
    'Shasta County': 'Condado de Shasta',
    'Sierra County': 'Condado de Sierra',
    'Merced County': 'Condado de Merced',
    'Lake County': 'Condado de Lake',
    'Contra Costa County': 'Condado de Contra Costa',
    'Solano County': 'Condado de Solano',
    'Santa Clara County': 'Condado de Santa Clara',
    'Butte County': 'Condado de Butte',
    'Ventura County': 'Condado de Ventura',
    'Sacramento County': 'Condado de Sacramento',
    'Mono County': 'Condado de Mono',
    'Orange County': 'Condado de Orange',
    'Calaveras County': 'Condado de Calaveras',
    'Del Norte County': 'Condado de Del Norte',
    'San Benito County': 'Condado de San Benito',
    'Trinity County': 'Condado de Trinity',
    'United States': 'Estados Unidos',
    'Kings County': 'Condado de Kings',
    'Stanislaus County': 'Condado de Stanislaus',
    'Plumas County': 'Condado de Plumas',
    'Mariposa County': 'Condado de Mariposa',
    'Inyo County': 'Condado de Inyo',
    'Riverside County': 'Condado de Riverside',
    'San Diego County': 'Condado de San Diego',
    'Colusa County': 'Condado de Colusa',
    'San Luis Obispo County': 'Condado de San Luis Obispo',
    'San Bernardino County': 'Condado de San Bernardino',
    'Yuba County': 'Condado de Yuba',
    'Napa County': 'Condado de Napa',
    'El Dorado County': 'Condado de El Dorado',
    'Glenn County': 'Condado de Glenn',
    'Monterey County': 'Condado de Monterey',
    'Santa Cruz County': 'Condado de Santa Cruz',
    'Yolo County': 'Condado de Yolo',
    'Amador County': 'Condado de Amador',
    'Tuolumne County': 'Condado de Tuolumne',
    'Lassen County': 'Condado de Lassen',
    'Modoc County': 'Condado de Modoc',
    'Los Angeles County': 'Condado de Los Angeles',
    'San Joaquin County': 'Condado de San Joaquin',
    'Alpine County': 'Condado de Alpine',
    'San Francisco County': 'Condado de San Francisco',
    'Alameda County': 'Condado de Alameda',
    'Tehama County': 'Condado de Tehama',
    'Tulare County': 'Condado de Tulare',
    'Sutter County': 'Condado de Sutter',
    'Imperial County': 'Condado de Imperial',
    'Placer County': 'Condado de Placer',
    'San Mateo County': 'Condado de San Mateo',
    'Mendocino County': 'Condado de Mendocino',
    'Madera County': 'Condado de Madera',
    'Sonoma County': 'Condado de Sonoma'
}


prop_dict = {
    'Total Government': 'Governo Total',
    'Total U.I. Covered': 'Total Coberto pelo Seguro Desemprego',
    'Federal Government': 'Governo Federal',
    'Local Government': 'Governo Local',
    'Total Covered': 'Total Coberto',
    'Private': 'Setor Privado',
    'State Government': 'Governo Estadual'
}

In [25]:
# Substituindo valores

df["Período de Tempo"] = df["Período de Tempo"].map(temp_dict)
df["Tipo de Área"] = df["Tipo de Área"].map(tipo_dict)
df["Nome da Área"] = df["Nome da Área"].map(nome_dict)
df["Propriedade"] = df["Propriedade"].map(prop_dict)

df_anual["Período de Tempo"] = df_anual["Período de Tempo"].map(temp_dict)
df_anual["Tipo de Área"] = df_anual["Tipo de Área"].map(tipo_dict)
df_anual["Nome da Área"] = df_anual["Nome da Área"].map(nome_dict)
df_anual["Propriedade"] = df_anual["Propriedade"].map(prop_dict)

In [ ]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

### *-- Feature Engineering -> Adicionando Coluna de Setores*

In [27]:
# Importando dicionário de outro arquivo para captar setores e subsetores
from pegarSetores import pegar_setores


setores = pegar_setores()

def aplicar_setor(linha):

    valorCell = linha["Nome da Indústria"]

    if valorCell in setores:
        return setores[valorCell]
    else:
        return "NÃO LISTADO"

In [28]:
# Aplicando a função para ambas as tabelas

df["Setor"] = df.apply(aplicar_setor, axis=1)
df_anual["Setor"] = df_anual.apply(aplicar_setor, axis=1)

In [ ]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

#### *--- Adicionando Setores Ainda Não Listados*

In [30]:
# Classificando por Código

codigos_setor = {
    "11": "Agriculture, Forestry, Fishing and Hunting",
    "21": "Mining, Quarrying, and Oil and Gas Extraction",
    "22": "Utilities",
    "23": "Construction",
    # "31-33": "Manufacturing",
    "31": "Manufacturing",
    "32": "Manufacturing",
    "33": "Manufacturing",
    "42": "Wholesale Trade",
    # "44-45": "Retail Trade",
    "44": "Retail Trade",
    "45": "Retail Trade",
    # "48-49": "Transportation and Warehousing",
    "48": "Transportation and Warehousing",
    "49": "Transportation and Warehousing",
    "51": "Information",
    "52": "Finance and Insurance",
    "53": "Real Estate and Rental and Leasing",
    "54": "Professional, Scientific, and Technical Services",
    "55": "Management of Companies and Enterprises",
    "56": "Administrative and Support and Waste Management and Remediation Services",
    "61": "Educational Services",
    "62": "Health Care and Social Assistance",
    "71": "Arts, Entertainment, and Recreation",
    "72": "Accommodation and Food Services",
    "81": "Other Services (except Public Administration)",
    "92": "Public Administration (not covered in economic census)"
}


In [ ]:
#Criando uma tabela temporária trimestral
temp_df = df[df["Setor"] == "NÃO LISTADO"].copy()
temp_df["Setor"] = temp_df["Código NAICS"].map(lambda cell: codigos_setor[cell.strip()[:2]] if cell.strip()[:2] in codigos_setor else "NÃO LISTADO")


#Criando uma tabela temporária anual
temp_df_anual = df_anual[df_anual["Setor"] == "NÃO LISTADO"].copy()
temp_df_anual["Setor"] = temp_df_anual["Código NAICS"].map(lambda cell: codigos_setor[cell.strip()[:2]] if cell.strip()[:2] in codigos_setor else "NÃO LISTADO")


display(temp_df.head())
print("\n-------------------\n")
display(temp_df_anual.head())

In [ ]:
df[df["Setor"] == "NÃO LISTADO"] = temp_df

df_anual[df_anual["Setor"] == "NÃO LISTADO"] = temp_df_anual

display(df.head())
print("\n-------------------\n")
display(df_anual.head())

## *-- Traduzindo Setores e Subsetores*

In [33]:
from deep_translator import GoogleTranslator

tradutor = GoogleTranslator(source="en", target="pt")

In [ ]:
# Traduzindo Subsetores

lista_subsetores = set(df["Nome da Indústria"])

tradutor = GoogleTranslator(source="en", target="pt")

for i, subsetor in enumerate(lista_subsetores):
    traduzido = tradutor.translate(subsetor)
    
    df.loc[df["Nome da Indústria"] == subsetor, "Nome da Indústria"] = traduzido
    df_anual.loc[df_anual["Nome da Indústria"] == subsetor, "Nome da Indústria"] = traduzido

    print(f"Linha {i + 1} traduzida")

In [ ]:
# Traduzindo Setores

lista_setores = set(df["Setor"])

tradutor = GoogleTranslator(source="en", target="pt")

for i, setor in enumerate(lista_setores):
    traduzido = tradutor.translate(setor)

    df.loc[df["Setor"] == setor, "Setor"] = traduzido
    df_anual.loc[df_anual["Setor"] == setor, "Setor"] = traduzido

    print(f"Linha {i + 1} traduzida")

## *-- Salvando Arquivos*

In [37]:
df.to_csv("Dados/Tratados/QCEW_Setorizado.csv", index=False, sep=";")
df_anual.to_csv("Dados/Tratados/QCEW_Anual_Setorizado.csv", index=False, sep=";")

# *- Loading na S3*

In [38]:
import os
import boto3
from dotenv import load_dotenv


# Carregar variáveis
load_dotenv(r"credentials/.env")

# Configure as cresdencias da AWS
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Crie um client S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Carregue o arquivo trimestral para o S3
s3.upload_file(
    Filename='Dados/Tratados/QCEW_Setorizado.csv',
    Bucket='projetofinal-consuelo',
    Key='QCEW_Setorizado.csv'
)

# Carregue o arquivo anual para o S3
s3.upload_file(
    Filename='Dados/Tratados/QCEW_Anual_Setorizado.csv',
    Bucket='projetofinal-consuelo',
    Key='QCEW_Anual_Setorizado.csv'
)
